In [1]:
import pandas as pd
import json

data = pd.read_csv("../dataset/stumbleupon.tsv", sep='\t')
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,24,0,5424,170,8,0.152941,0.079130,0,IBM Sees Holographic Calls Air Breathing Batte...,A sign stands outside the International Busine...
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,40,0,4973,187,9,0.181818,0.125448,1,The Fully Electronic Futuristic Starting Gun T...,And that can be carried on a plane without the...
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight th...,Apples The most popular source of antioxidants...
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,24,0,2737,120,5,0.041667,0.100858,1,10 Foolproof Tips for Better Sleep,There was a period in my life when I had a lot...
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,14,0,12032,162,10,0.098765,0.082569,0,The 50 Coolest Jerseys You Didn t Know Existed...,Jersey sales is a curious business Whether you...


## Predicting "Greenness" Of Content

This dataset comes from [stumbleupon](https://www.stumbleupon.com/), a web page recommender.  

A description of the columns is below

FieldName|Type|Description
---------|----|-----------
url|string|Url of the webpage to be classified
title|string|Title of the article
body|string|Body text of article
urlid|integer| StumbleUpon's unique identifier for each url
boilerplate|json|Boilerplate text
alchemy_category|string|Alchemy category (per the publicly available Alchemy API found at www.alchemyapi.com)
alchemy_category_score|double|Alchemy category score (per the publicly available Alchemy API found at www.alchemyapi.com)
avglinksize| double|Average number of words in each link
commonlinkratio_1|double|# of links sharing at least 1 word with 1 other links / # of links
commonlinkratio_2|double|# of links sharing at least 1 word with 2 other links / # of links
commonlinkratio_3|double|# of links sharing at least 1 word with 3 other links / # of links
commonlinkratio_4|double|# of links sharing at least 1 word with 4 other links / # of links
compression_ratio|double|Compression achieved on this page via gzip (measure of redundancy)
embed_ratio|double|Count of number of <embed> usage
frameBased|integer (0 or 1)|A page is frame-based (1) if it has no body markup but have a frameset markup
frameTagRatio|double|Ratio of iframe markups over total number of markups
hasDomainLink|integer (0 or 1)|True (1) if it contains an <a> with an url with domain
html_ratio|double|Ratio of tags vs text in the page
image_ratio|double|Ratio of <img> tags vs text in the page
is_news|integer (0 or 1) | True (1) if StumbleUpon's news classifier determines that this webpage is news
lengthyLinkDomain| integer (0 or 1)|True (1) if at least 3 <a> 's text contains more than 30 alphanumeric characters
linkwordscore|double|Percentage of words on the page that are in hyperlink's text
news_front_page| integer (0 or 1)|True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
non_markup_alphanum_characters|integer| Page's text's number of alphanumeric characters
numberOfLinks|integer Number of <a>|markups
numwords_in_url| double|Number of words in url
parametrizedLinkRatio|double|A link is parametrized if it's url contains parameters or has an attached onClick event
spelling_errors_ratio|double|Ratio of words not found in wiki (considered to be a spelling mistake)
label|integer (0 or 1)|User-determined label. Either evergreen (1) or non-evergreen (0); available for train.tsv only

> ### Let's try extracting some of the text content.
> ### Create a feature for the title containing 'recipe'. Is the % of evegreen websites higher or lower on pages that have recipe in the the title?

In [2]:
# Option 1: Create a function to check for this

def has_recipe(text_in):
    try:
        if 'recipe' in str(text_in).lower():
            return 1
        else:
            return 0
    except: 
        return 0
        
data['recipe'] = data['title'].map(has_recipe)

# Option 2: lambda functions

#data['recipe'] = data['title'].map(lambda t: 1 if 'recipe' in str(t).lower() else 0)


# Option 3: string functions
data['recipe'] = data['title'].str.contains('recipe')

 ### Demo: Use of the Count Vectorizer

In [19]:
titles = data['body'].fillna('')

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features = 1000, 
                             ngram_range=(1, 2), 
                             stop_words='english',
                             binary=True)

# Use `fit` to learn the vocabulary of the titles
#vectorizer.fit(titles)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
#X = vectorizer.transform(titles)

# LDA in Gensim

docs = vectorizer.fit_transform(data.body.dropna())
# Build a mapping of numerical ID to word
id2word = dict(enumerate(vectorizer.get_feature_names()))
id2word


{0: u'000',
 1: u'10',
 2: u'10 minutes',
 3: u'100',
 4: u'11',
 5: u'12',
 6: u'13',
 7: u'14',
 8: u'15',
 9: u'15 minutes',
 10: u'16',
 11: u'17',
 12: u'18',
 13: u'19',
 14: u'20',
 15: u'20 minutes',
 16: u'2008',
 17: u'2009',
 18: u'2010',
 19: u'2011',
 20: u'2012',
 21: u'21',
 22: u'22',
 23: u'23',
 24: u'24',
 25: u'25',
 26: u'26',
 27: u'28',
 28: u'30',
 29: u'30 minutes',
 30: u'35',
 31: u'350',
 32: u'350 degrees',
 33: u'40',
 34: u'400',
 35: u'45',
 36: u'50',
 37: u'60',
 38: u'ability',
 39: u'able',
 40: u'absolutely',
 41: u'according',
 42: u'actually',
 43: u'adapted',
 44: u'add',
 45: u'added',
 46: u'adding',
 47: u'addition',
 48: u'additional',
 49: u'advice',
 50: u'age',
 51: u'ago',
 52: u'ahead',
 53: u'air',
 54: u'allow',
 55: u'alternative',
 56: u'amazing',
 57: u'america',
 58: u'american',
 59: u'apart',
 60: u'apple',
 61: u'area',
 62: u'aren',
 63: u'art',
 64: u'article',
 65: u'aside',
 66: u'ask',
 67: u'asked',
 68: u'attention',
 69:

In [22]:
from gensim.models.ldamodel import LdaModel
from gensim.matutils import Sparse2Corpus

# First we convert our word-matrix into gensim's format 
corpus = Sparse2Corpus(docs, documents_columns = False)
# Then we fit an LDA model
lda_model = LdaModel(corpus=corpus, id2word=id2word, num_topics=15)
lda_model

In [24]:
# Evaluating the Fit

lda_model.show_topics?

num_topics = 15
num_words_per_topic = 5
for ti, topic in enumerate(lda_model.show_topics(num_topics = num_topics, num_words = num_words_per_topic)):
    print ("Topic: %d" % (ti))
    print (topic)
    print ()

Topic: 0
(0, u'0.010*"left" + 0.010*"like" + 0.008*"text" + 0.008*"make" + 0.008*"size"')
()
Topic: 1
(1, u'0.012*"com" + 0.010*"new" + 0.008*"like" + 0.008*"video" + 0.008*"just"')
()
Topic: 2
(2, u'0.008*"health" + 0.007*"people" + 0.006*"help" + 0.006*"like" + 0.006*"body"')
()
Topic: 3
(3, u'0.007*"use" + 0.007*"make" + 0.006*"like" + 0.006*"just" + 0.006*"way"')
()
Topic: 4
(4, u'0.009*"add" + 0.008*"heat" + 0.007*"minutes" + 0.007*"recipe" + 0.006*"cups"')
()
Topic: 5
(5, u'0.012*"food" + 0.011*"recipes" + 0.009*"healthy" + 0.008*"recipe" + 0.007*"eating"')
()
Topic: 6
(6, u'0.007*"like" + 0.007*"just" + 0.007*"new" + 0.007*"year" + 0.006*"world"')
()
Topic: 7
(7, u'0.007*"recipe" + 0.007*"medium" + 0.007*"garlic" + 0.007*"pepper" + 0.007*"oil"')
()
Topic: 8
(8, u'0.010*"don" + 0.009*"like" + 0.009*"just" + 0.007*"really" + 0.007*"way"')
()
Topic: 9
(9, u'0.016*"food" + 0.013*"health" + 0.011*"com" + 0.010*"sign" + 0.010*"community"')
()
Topic: 10
(10, u'0.008*"minutes" + 0.008*"

In [26]:

from gensim.models.word2vec import Word2Vec
# Setup the body text
text = data.body.dropna().map(lambda x: x.split())

from gensim.models import Word2Vec
model = Word2Vec(text, size=100, window=5, min_count=5, workers=4)
model




In [27]:
model.most_similar(positive=['cookie', 'brownie'])

[(u'cupcake', 0.9251306056976318),
 (u'pie', 0.851150631904602),
 (u'cake', 0.8485327363014221),
 (u'crust', 0.8482750058174133),
 (u'tart', 0.8394856452941895),
 (u'cheesecake', 0.8366951942443848),
 (u'candy', 0.8359284400939941),
 (u'mini', 0.834355354309082),
 (u'icing', 0.822581946849823),
 (u'frosting', 0.820372462272644)]

In [34]:
model.most_similar(positive=['unhealthy'])

[(u'carbohydrates', 0.8275607824325562),
 (u'enzymes', 0.8095883131027222),
 (u'nutrient', 0.793606698513031),
 (u'nutritional', 0.7849339246749878),
 (u'naturally', 0.7845566868782043),
 (u'beneficial', 0.7815964221954346),
 (u'supplements', 0.7756664156913757),
 (u'proteins', 0.7695953845977783),
 (u'essential', 0.7673078775405884),
 (u'starches', 0.7655875086784363)]

 ### Demo: Build a random forest model to predict evergreeness of a website using the title features

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators = 20)
    
# Use `fit` to learn the vocabulary of the titles
vectorizer.fit(titles)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(titles).toarray()
y = data['label']

from sklearn.cross_validation import cross_val_score

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

/anaconda/envs/py27_ds_dt_13/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


CV AUC [ 0.78866673  0.80274934  0.80443387], Average AUC 0.798616646632


### Exercise: Build a random forest model to predict evergreeness of a website using the title features and quantitative features

In [5]:
# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X_text_features = vectorizer.transform(titles)

# Identify the features you want from the original dataset
other_features_columns = ['html_ratio', 'image_ratio']
other_features = data[other_features_columns]

# Stack them horizontally together
# This takes all of the word/n-gram columns and appends on two more columns for `html_ratio` and `image_ratio`
from scipy.sparse import hstack
X = hstack((X_text_features, other_features)).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

# What features of these are most important?
model.fit(X, y)

all_feature_names = vectorizer.get_feature_names() + other_features_columns
feature_importances = pd.DataFrame({'Features' : all_feature_names, 'Importance Score': model.feature_importances_})
feature_importances.sort_values('Importance Score', ascending=False).head()

CV AUC [ 0.78537849  0.79580237  0.788348  ], Average AUC 0.789842952576


,Features,Importance Score
1000,html_ratio,0.155853
1001,image_ratio,0.095620
715,recipe,0.039011
721,recipes,0.020534
183,chicken,0.012650


 ### Exercise: Build a random forest model to predict evergreeness of a website using the body features

In [6]:
body_text = data['body'].fillna('')

# Use `fit` to learn the vocabulary
vectorizer.fit(body_text)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.83658603  0.84479776  0.83873979], Average AUC 0.840041195125


 ### Exercise: Use `TfIdfVectorizer` instead of `CountVectorizer` - is this an improvement?

In [7]:
titles = data['title'].fillna('')

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features = 1000, 
                             ngram_range=(1, 2), 
                             stop_words='english')


# Use `fit` to learn the vocabulary
vectorizer.fit(body_text)

# Use `tranform` to generate the sample X word matrix - one column per feature (word or n-grams)
X = vectorizer.transform(body_text).toarray()

scores = cross_val_score(model, X, y, scoring='roc_auc')
print('CV AUC {}, Average AUC {}'.format(scores, scores.mean()))

CV AUC [ 0.84268957  0.85274835  0.8393846 ], Average AUC 0.844940841904
